In [1]:
import pandas as pd
import numpy as np

# Adapted from https://github.com/MeMartijn/FakeNewsDetection
from data_loader import DataLoader  
from classifiers import Classifiers

In [2]:
data = DataLoader()

/Users/martijn/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
bert = data.get_bert()

In [4]:
bert

,name,type,embedding
0,meta file,class,"[[[1.7955303, -0.39878398, 0.8161868, -0.49417..."
1,meta source,class,"[[[1.63934, -0.27256083, 1.0740907, -0.5459729..."
2,property,class,"[[[0.63757765, -0.021747919, 0.14703423, -0.97..."
3,set,class,"[[[0.45664948, 0.22363535, 0.6579699, -0.63327..."
4,item,class,"[[[0.4551617, -0.6158074, 0.12454513, -0.97725..."
...,...,...,...
498455,volume,attribute,"[[[-0.1947565, 0.18598285, 0.2679827, -0.69112..."
498456,unit,attribute,"[[[-0.237941, 0.035174966, -0.36673543, -1.065..."
498457,item name,attribute,"[[[0.8360246, -0.51640177, 0.71521986, -0.8962..."
498458,std volume,attribute,"[[[0.08100525, -0.8526971, 0.22229162, -0.4153..."


In [20]:
pooled_df = data.apply_pooling('max', bert[['embedding', 'type']])

Applying max pooling to the dataset...


/Users/martijn/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from hypopt import GridSearch

In [23]:
X_train, X_test, y_train, y_test = train_test_split(pooled_df['embedding'].values, pooled_df['type'].values, test_size=0.25, random_state=0)

In [ ]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
gs = GridSearch(model = LogisticRegression(penalty = 'l2'), param_grid = param_grid)
gs.fit(X_train, y_train)

score = gs.score(X_test, y_test)
score